<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/crawler/new_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import html
import re

In [2]:
def get_new_code(id):
  url = 'http://www.38.co.kr/html/fund/?o=v&no=' + str(id) + '&l='
  result = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'})
  bs_obj = BeautifulSoup(result.content, "html.parser")

  # 기업개요
  table1 = bs_obj.find_all('table', {'summary':'기업개요'})[0]
  d = [x.get_text() for x in table1.find_all('td')]
  row1 = [x.replace(' ', '') for x in d[::2]]

  elements1 = d[1::2]
  remove_chars = ['\xa0', '\t', '/']
  for char in remove_chars:
    elements1 = [x.replace(char, ' ') for x in elements1]
  elements1 = [x.strip() for x in elements1]


  # 공모정보
  table2 = bs_obj.find_all('table', {'summary':'공모정보'})[0]

  d = [x.get_text() for x in table2.find_all('td')]
  row2 = [x.replace(' ', '') for x in d[::2][:-1]]
  row2.remove('상장공모')

  elements2 = d[1::2]
  remove_chars = ['\xa0', ' ', '\t', '/']
  for char in remove_chars:
    elements2 = [x.replace(char, '') for x in elements2]
  ex = [x.split(':') for x in elements2.pop(2).split('\n')]

  for i in ex:
    try:
      row2.append(i[0])
      elements2.append(i[1])
    except:
      pass

  row = row1 + row2
  elements = elements1 + elements2
  
  return dict(zip(row, elements))

In [3]:
'''from multiprocessing import Pool

dic = {}
with Pool(500) as p:
    dic = p.map(get_new_code, [x for x in range(1700,1,-1)])

df = pd.DataFrame()
for d in dic:
  df = df.append(d, ignore_index=True)'''

'from multiprocessing import Pool\n\ndic = {}\nwith Pool(500) as p:\n    dic = p.map(get_new_code, [x for x in range(1700,1,-1)])\n\ndf = pd.DataFrame()\nfor d in dic:\n  df = df.append(d, ignore_index=True)'

In [4]:
df = pd.DataFrame()
for i in range(0, 1700):
  try:
    df = df.append(get_new_code(i), ignore_index=True)
  except:
    pass

In [5]:
df.to_csv('new_stocks.csv', encoding='cp949')

from google.colab import files
files.download('new_stocks.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
df

,공모금액,기업구분,대표자,대표전화,매출액,법인세비용차감전계속사업이익,본점소재지,순이익,시장구분,신주모집,액면가,업종,자본금,종목명,종목코드,주간사,진행상황,청약경쟁률,총공모주식수,최대주주,홈페이지,확정공모가,희망공모가액,구주매출
0,13(백만원),중소벤쳐,하재홍,02-2107-5793,"20,979 (백만원)","5,535 (백만원)",서울 금천구 가산동 459-40,"4,751 (백만원)",코스닥,"2,100,000주(100%)",500원,그외 기타 조립금속제품 제조업,"4,200 (백만원)",아이레보,072430,교보증권\n,신규상장,,"2,100,000주",하재홍외 1명 ( 44.37 % ),,6원,"5,000~6,500원",NaN
1,62(백만원),중소벤쳐,양덕준,02-3019-1700,"79,963 (백만원)","8,663 (백만원)",서울 서초구 서초동 1586-7,"8,088 (백만원)",코스닥,"1,320,000주(100%)",500원,"방송수신기 및 기타 영상, 음향기기 제조업","2,590 (백만원)",레인콤,060570,현대증권\n,신규상장,,"1,320,000주",양덕준외 3명 ( 43.31 % ),www.iriver.co.kr,47원,"37,000~47,000원",NaN
2,5(백만원),중소벤쳐,권오현,02-3430-5002,"17,761 (백만원)","2,238 (백만원)",서울시 강남구 대치동 889-11 대치빌딩 10층,"1,805 (백만원)",코스닥,"1,340,000주(100%)",500원,일반 영화 및 비디오 제작업,"2,030 (백만원)",비트윈,066410,미래에셋\n,신규상장,,"1,340,000주",권오현외 3명 ( 57.94 % ),www.towin-global.com,4원,"3,500~4,500원",NaN
3,33(백만원),,,,- (백만원),0 (백만원),,- (백만원),거래소,"2,779,800주(100%)",500원,"기타 소프트웨어 자문, 개발 및 공급업",- (백만원),텔코웨어,078000,삼성증권\n,,45.72:1,"2,779,800주",,,12원,"12,000~15,500원",NaN
4,15(백만원),중소벤쳐,박성찬,02)6004-1089,"46,656 (백만원)","5,986 (백만원)",서울 강남구 역삼1동 로담코빌딩 11,"5,103 (백만원)",코스닥,"1,840,000주(100%)",500원,데이터베이스 및 온라인 정보제공업,"2,145 (백만원)",다날,064260,LG증권\n,신규상장,68.25:1,"1,840,000주",박성찬외 2명 ( 37.79 % ),www.danal.co.kr,8원,"10,000~12,000원",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,"40,000(백만원)",중소일반,곽대환,,- (백만원),0 (백만원),서울특별시 영등포구 여의대로 108,- (백만원),코스닥,"20,000,000주(100%)",100원,기타 금융업,550 (백만원),NH스팩20호,391060,NH투자증권\n,공모주,,"20,000,000주",-,,-원,"2,000~2,000원",NaN
1296,"10,000(백만원)",중소일반,이현식,,- (백만원),0 (백만원),서울특별시 중구 삼일대로 343,- (백만원),코스닥,"5,000,000주(100%)",100원,금융 지원 서비스업,46 (백만원),대신밸런스스팩10호,387310,대신증권\n,공모주,,"5,000,000주",-,,-원,"2,000~2,000원",NaN
1297,"12,500(백만원)",중소일반,김종환,,- (백만원),0 (백만원),서울특별시 영등포구 여의대로 70 신한금융투자 4층,- (백만원),코스닥,"6,250,000주(100%)",100원,금융 지원 서비스업,110 (백만원),신한스팩8호,393360,신한금융투자\n,공모주,,"6,250,000주",-,,-원,"2,000~2,000원",NaN
1298,"43,450(백만원)",중소일반,염정선,031-737-8208,78 (백만원),"-6,240 (백만원)","경기 성남시 중원구 둔촌대로 560, 406,407호 (상대원동,벽산테크노피아아파트...","-6,240 (백만원)",코스닥,"3,950,000주(100%)",500원,의약품 제조업,"11,194 (백만원)",차백신연구소,261780,삼성증권\n,공모주,,"3,950,000주",차바이오텍 46%,www.chavaccine.com,-원,"11,000~15,000원",NaN
